In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import Markdown as md


tqdm.pandas()
regions = pd.read_json('https://geo.api.gouv.fr/regions').rename(columns={'code': 'codeRegion'})
departements = regions.merge(pd.read_json('https://geo.api.gouv.fr/departements'), 
                             on='codeRegion', suffixes= ('Region', ''))

In [2]:
departements

,nomRegion,codeRegion,nom,code
0,Guadeloupe,1,Guadeloupe,971
1,Martinique,2,Martinique,972
2,Guyane,3,Guyane,973
3,La Réunion,4,La Réunion,974
4,Mayotte,6,Mayotte,976
...,...,...,...,...
96,Provence-Alpes-Côte d'Azur,93,Bouches-du-Rhône,13
97,Provence-Alpes-Côte d'Azur,93,Var,83
98,Provence-Alpes-Côte d'Azur,93,Vaucluse,84
99,Corse,94,Corse-du-Sud,2A


## Données UDIs et prélèvements (data.gouv.fr)

In [3]:
# Dataframe des UDIs pour chaque année
UDIs_data_gouv = {}
for file in glob.glob('data/DIS_20*/DIS_COM_UDI_*'):
    data = pd.read_csv(file)
    year = int(file.split('_')[-1].strip('.txt'))
    UDIs_data_gouv[year] = data

In [4]:
print(UDIs_data_gouv[2016].shape)
UDIs_data_gouv[2016].head()

(44994, 6)


,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07


In [5]:
# Dataframe des prélèvements pour chaque année. Uniquement pour avoir le numéro de département du réseau
PLVs_data_gouv = {}
for file in glob.glob('data/DIS_20*/DIS_PLV_*'):
    data = pd.read_csv(file, dtype={'cdreseau': str, 'cddept': str}, 
                       usecols=['cdreseau', 'cddept'])
    year = int(file.split('_')[-1].strip('.txt'))
    PLVs_data_gouv[year] = data

In [6]:
print(PLVs_data_gouv[2016].shape)
PLVs_data_gouv[2016].head()

(396036, 2)


,cddept,cdreseau
0,001,001000003
1,001,001000003
2,001,001000003
3,001,001000003
4,001,001000003


In [7]:
# Fonctions pour homogéneiser les champs des tableaux

def homog_udi(udi):
    return '0'*(len(udi) - 9) + udi

def homog_comm(comm):
    while len(comm) < 5:
        comm= '0'+ comm
    return comm

def homog_dept(dept):
    return '0'*(3 - len(dept)) + dept

def homog_prel(prel):
    while len(prel) < 11:
        prel= '0' + prel
    return prel


In [8]:
## Homgéneiser les UDI et les departements
for k, v in PLVs_data_gouv.items():
    v['cdreseau'] = v['cdreseau'].progress_apply(homog_udi)
    v['cddept'] = v['cddept'].progress_apply(homog_dept)
    # v['cdreseau'] = v['cdreseau'].apply(homog_udi)
    # v['cddept'] = v['cddept'].apply(homog_dept)

departements['code'] = departements['code'].apply(homog_dept)

100%|██████████| 427020/427020 [00:00<00:00, 1059153.40it/s]


In [9]:
# jointure entre UDIs et prélèvements
join_data_gouv = {k: UDIs_data_gouv[k].merge(v.drop_duplicates('cdreseau'), on='cdreseau', how='left')
                  for k, v in PLVs_data_gouv.items()}
# join_data_gouv = {k: UDIs_data_gouv[k].merge(v, on='cdreseau') 
#                   for k, v in PLVs_data_gouv.items()}

# join_data_gouv = UDIs_data_gouv

join_data_gouv[2016]

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,001
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,001
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,001
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,001
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,001
...,...,...,...,...,...,...,...
44989,97617,TSINGONI,MIRERENI VILLAGE,976000059,TSINGONI COMBANI,2013-05-30,976
44990,97617,TSINGONI,MROUALE VILLAGE,976000059,TSINGONI COMBANI,2013-05-30,976
44991,97617,TSINGONI,COMBANI VILLAGE,976000059,TSINGONI COMBANI,2013-05-30,976
44992,97701,SAINT-BARTHELEMY,Toute la commune,971000159,SAINT-BARTHELEMY,2010-10-19,971


# Jointure UDIs data.gouv.fr et AtlaSante, par région

In [10]:
data_AtlaSante = pd.read_csv('data/info_AtlaSante.csv').set_index('Région')
data_AtlaSante

,Année,Champ AtlaSante,Champ data.gouv,Fichier
Région,,,,
Auvergne-Rhône-Alpes,2020,c_code_udi,cdreseau,data/AtlaSante_Auvergne_Rhone_Alpes_2020/ars_r...
Bourgogne-Franche-Comté,2019,code,cdreseau,data/AtlaSante_Bourgogne_Franche_Comte_2019/ar...
Bretagne,2017,c_ins_code,cdreseau,data/AtlaSante_Bretagne_2017/ars_r53_bretagne_...
Centre-Val de Loire,2020,code,cdreseau,data/AtlaSante_Centre_Val_de_Loire_2020/ars_r2...
Corse,2017,c_cn_udi,cdreseau,data/AtlaSante_Corse_2017/ars_r94_udi_j.json
Normandie,2020,codeudi,cdreseau,data/AtlaSante_Normandie_2020/ars_r28_udi_2020...
Nouvelle-Aquitaine,2018,c_code_udi,cdreseau,data/AtlaSante_Nouvelle-Aquitaine_2018/ars_r75...
Occitanie,2020,code_n_udi,cdreseau,data/AtlaSante_Occitanie_2020/ars_r76_udi_2020...
Provence-Alpes-Côte d'Azur,2020,code_infof,cdreseau,data/AtlaSante_PACA_2019/ars_r93_udi_infofactu...


In [11]:
# Load maps. Remove duplicates and empty geometry rows
cartes_AtlaSante = {}
for index, row in tqdm(data_AtlaSante.iterrows(), total=data_AtlaSante.shape[0]):
    column = row['Champ AtlaSante']
    cartes_AtlaSante[index] = gpd.read_file(row['Fichier']).dropna(subset=['geometry'])\
                               .drop_duplicates(column)

100%|██████████| 11/11 [00:30<00:00,  2.80s/it]


In [12]:
for region, data in data_AtlaSante.iterrows():
    # if 'Provence' not in region: 
    #     continue
    
    df = cartes_AtlaSante[region]
    data_gouv = join_data_gouv[data['Année']].drop_duplicates(data['Champ data.gouv'])

    # restrict to departments in the region
    depts = departements.set_index('nomRegion').loc[region if region != 'Réunion' else 'La Réunion', 'code']
    data_gouv_dept = data_gouv.loc[data_gouv['cddept'].isin([depts] if isinstance(depts, (int, str)) else depts)]
    
    join = df.merge(data_gouv_dept, 
                    left_on=data['Champ AtlaSante'], 
                    right_on=data['Champ data.gouv'],
                    how='outer')
    
    
    match = join.dropna(subset=(data['Champ AtlaSante'], data['Champ data.gouv']))
    union = join.dropna(subset=(data['Champ AtlaSante'], data['Champ data.gouv']), how='all')
    pas_AtlasSante = join[join[data['Champ AtlaSante']].isna()]
    pas_dataGouv = join[join[data['Champ data.gouv']].isna()]
    pas_dept = df[~df[data['Champ AtlaSante']].isin(match[data['Champ AtlaSante']])].\
               merge(data_gouv, left_on=data['Champ AtlaSante'], right_on=data['Champ data.gouv'])
    
    data_AtlaSante.loc[region, 'UDIs AtlasSante'] = df.shape[0]
    data_AtlaSante.loc[region, 'UDIs data.gouv'] = join[data['Champ data.gouv']].notna().sum()
    data_AtlaSante.loc[region, 'Union'] = union.shape[0]
    data_AtlaSante.loc[region, 'Intersection'] = match.shape[0]
    data_AtlaSante.loc[region, 'Pas dans AtlaSante'] = pas_AtlasSante.shape[0]
    data_AtlaSante.loc[region, 'Pas dans data.gouv'] = pas_dataGouv.shape[0]
    data_AtlaSante.loc[region, 'Pas dans dept'] = pas_dept.shape[0]
    
    display(md(f"### {region} {data['Année']}"))
    display(match)
    display(pas_AtlasSante)
    display(pas_dataGouv)
    display(pas_dept)
    
    match.to_csv(f"outputs/{region}_{data['Année']}_intersection.csv")
    pas_AtlasSante[data_gouv_dept.columns].to_csv(f"outputs/{region}_{data['Année']}_pasAtlaSante.csv")
    pas_dataGouv[df.columns].to_csv(f"outputs/{region}_{data['Année']}_pasDataGouv.csv")                 

### Auvergne-Rhône-Alpes 2020

,gid,c_gid,c_code_udi,t1_gid,t1_udicode,t1_udinom,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
0,1.0,1.0,003000300,318.0,003000300,RESEAU AINAY-LE-CHATEAU,"MULTIPOLYGON (((2.73353 46.69814, 2.73359 46.6...",03003,AINAY-LE-CHATEAU,-,003000300,RESEAU AINAY-LE-CHATEAU,2010-09-18,003
1,2.0,2.0,003000301,319.0,003000301,RESEAU CLAUDE DECLOITRE,"MULTIPOLYGON (((3.42878 46.06524, 3.42865 46.0...",03001,ABREST,-,003000301,RESEAU CLAUDE DECLOITRE,2013-01-09,003
2,3.0,3.0,003000304,321.0,003000304,RESEAU CERILLY,"MULTIPOLYGON (((2.82875 46.58549, 2.82816 46.5...",03048,CERILLY,-,003000304,RESEAU CERILLY,2010-09-18,003
3,4.0,4.0,003000305,322.0,003000305,RESEAU LA CHABANNE,"MULTIPOLYGON (((3.71546 46.01416, 3.71597 46.0...",03050,CHABANNE (LA),"La Roffière, Brunard,",003000305,RESEAU LA CHABANNE,2010-09-18,003
4,5.0,5.0,003000317,327.0,003000317,RESEAU COMMENTRY,"MULTIPOLYGON (((2.77439 46.26244, 2.73181 46.2...",03082,COMMENTRY,Centre,003000317,RESEAU COMMENTRY,2010-09-18,003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5392,5464.0,5464.0,007000637,613.0,007000637,ILE DE L EYRIEUX,"MULTIPOLYGON (((4.71239 44.79843, 4.71253 44.7...",07237,SAINT-FORTUNAT-SUR-EYRIEUX,en totalité,007000637,ILE DE L EYRIEUX,2017-01-01,007
5393,5465.0,5465.0,007001407,812.0,007001407,LAMASTRE UDI VILLE,"MULTIPOLYGON (((4.57715 45.00267, 4.57727 45.0...",07129,LAMASTRE,"bourg centre, gouteneyre route tournon",007001407,LAMASTRE UDI VILLE,2010-09-07,007
5394,5466.0,5466.0,007002656,926.0,007002656,UDI SAUVANS,"MULTIPOLYGON (((4.47055 45.04089, 4.46808 45.0...",07080,DEVESSET,Fontaneyre,007002656,UDI SAUVANS,2020-02-06,007
5396,5468.0,5468.0,007001054,727.0,007001054,PRANLES LA CHARRIERE,"MULTIPOLYGON (((4.53872 44.78412, 4.53974 44.7...",07184,PRANLES,Pivernet - La sagnole - La charrière,007001054,PRANLES LA CHARRIERE,2010-09-07,007


,gid,c_gid,c_code_udi,t1_gid,t1_udicode,t1_udinom,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
5399,NaN,NaN,NaN,NaN,NaN,NaN,None,03163,MARIOL,Les Plantés-Le Fréty,003000335,RESEAU VAL. SICHON - MARIOL,2010-09-18,003
5400,NaN,NaN,NaN,NaN,NaN,NaN,None,07018,ASTET,HAMEAU DE SEDASSIER,007006178,ASTET SEDASSIER,2020-07-28,007
5401,NaN,NaN,NaN,NaN,NaN,NaN,None,07159,MIRABEL,"Bourg centre, Les Granges",007005705,SI ODS MIRABEL,2018-10-04,007
5402,NaN,NaN,NaN,NaN,NaN,NaN,None,15008,ANTIGNAC,URLANDE,015000432,URLANDE PAR TTP COMPIER HS,2010-09-18,015
5403,NaN,NaN,NaN,NaN,NaN,NaN,None,15049,CHEYLADE,-,015001019,SOUBREROCHE HS,2010-09-18,015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5484,NaN,NaN,NaN,NaN,NaN,NaN,None,74286,VACHERESSE,-,074006628,VACHERESSE- UBINE (HAMEAU),2014-04-07,074
5485,NaN,NaN,NaN,NaN,NaN,NaN,None,74310,VIUZ-LA-CHIESAZ,-,074003224,SEMNOZ RESTAURANT CHATILLON,2010-09-07,074
5486,NaN,NaN,NaN,NaN,NaN,NaN,None,74310,VIUZ-LA-CHIESAZ,-,074003231,SEMNOZ ACCUEIL ALPIN,2010-09-07,074
5487,NaN,NaN,NaN,NaN,NaN,NaN,None,74310,VIUZ-LA-CHIESAZ,-,074003232,SEMNOZ CHALET NORDIQUE FORET,2010-09-07,074


,gid,c_gid,c_code_udi,t1_gid,t1_udicode,t1_udinom,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
154,156.0,156.0,015002777,NaN,None,None,"MULTIPOLYGON (((3.29830 44.99825, 3.31080 44.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,212.0,212.0,015000862,NaN,None,None,"MULTIPOLYGON (((3.00230 44.82287, 3.00881 44.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,277.0,277.0,015004482,NaN,None,None,"MULTIPOLYGON (((2.77252 45.24549, 2.77291 45.2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,287.0,287.0,015002820,NaN,None,None,"MULTIPOLYGON (((2.95058 44.80954, 2.95091 44.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,288.0,288.0,015002816,NaN,None,None,"MULTIPOLYGON (((2.95069 44.80911, 2.95149 44.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5348,5419.0,5419.0,007003491,NaN,None,None,"MULTIPOLYGON (((4.75676 44.62519, 4.75203 44.6...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5359,5430.0,5430.0,007005578,NaN,None,None,"MULTIPOLYGON (((4.24901 44.78797, 4.25202 44.7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5390,5462.0,5462.0,007001630,865.0,007001630,SI HTE-AL LAVEYRUNE LA BASTIDE,"MULTIPOLYGON (((3.90502 44.59933, 3.91086 44.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5395,5467.0,5467.0,007006185,978.0,007006185,LAFARRE TETE COIRET,"MULTIPOLYGON (((4.49832 45.05682, 4.49756 45.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


,gid,c_gid,c_code_udi,t1_gid,t1_udicode,t1_udinom,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept


...

### Réunion 2019

,gid,cn_udi,locked,_userid_creation,_userid_modification,_edit_datemaj,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
0,1.0,974000003,None,None,None,None,"MULTIPOLYGON (((55.34353 -21.22198, 55.34413 -...",97401,AVIRONS (LES),"Bader, Bellecombe, Toupin (B1)",974000003,RESEAU TOUPIN-RUISSEAU,2010-10-08,974
1,2.0,974000004,None,None,None,None,"MULTIPOLYGON (((55.34082 -21.23845, 55.34073 -...",97401,AVIRONS (LES),"Les Bananes, Bois Blanc, La Croix, Brûlé, Ruis...",974000004,RESEAU AVIRONS VILLE-BANANES,2019-08-02,974
2,3.0,974000005,None,None,None,None,"MULTIPOLYGON (((55.34596 -21.23831, 55.34833 -...",97401,AVIRONS (LES),"Dozinval, Ravine Sèche (D3)",974000005,RESEAU RAVINE SECHE AVIRONS,2010-10-08,974
3,4.0,974000007,None,None,None,None,"MULTIPOLYGON (((55.47560 -21.12741, 55.47731 -...",97424,CILAOS,"La Plateforme, Mare Sèche, Petit Matharum, Brû...",974000007,RESEAU CILAOS VILLE,2019-08-02,974
4,5.0,974000009,None,None,None,None,"MULTIPOLYGON (((55.47382 -21.16327, 55.47531 -...",97424,CILAOS,Palmiste Rouge,974000009,RESEAU PALMISTE ROUGE,2010-10-08,974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,170.0,974003621,None,None,None,None,"MULTIPOLYGON (((55.32288 -20.94545, 55.32319 -...",97407,PORT (LE),"ZAC 2000, Zone pénitencière",974003621,RESEAU LE PORT PICHETTE,2015-04-27,974
171,172.0,974003670,None,None,None,None,"MULTIPOLYGON (((55.52737 -21.05358, 55.52659 -...",97421,SALAZIE,Mare à Poule d'eau,974003670,RESEAU MARE A POULE D'EAU,2016-06-24,974
172,173.0,974003834,None,None,None,None,"MULTIPOLYGON (((55.70424 -21.01965, 55.70434 -...",97410,SAINT-BENOIT,Centre ville,974003834,RESEAU VILLE ST-BENOIT,2018-07-18,974
173,174.0,974000049,None,None,None,None,"MULTIPOLYGON (((55.63926 -20.97805, 55.63838 -...",97409,SAINT-ANDRE,"Lotissement Dioré, Sarabé",974000049,RESEAU PETIT DIORE,2019-08-26,974


,gid,cn_udi,locked,_userid_creation,_userid_modification,_edit_datemaj,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
175,NaN,NaN,NaN,NaN,NaN,NaN,None,97404,ETANG-SALE (L'),Les Canots Les Bas,974000019,RESEAU LES CANOTS LES BAS,2014-01-13,974
176,NaN,NaN,NaN,NaN,NaN,NaN,None,97405,PETITE-ILE,"Anse Les Bas, Piton du Calvaire",974000764,RESEAU PETITE ILE VILLE,2019-08-26,974
177,NaN,NaN,NaN,NaN,NaN,NaN,None,97411,SAINT-DENIS,Brulé les Bas,974000062,RESEAU BRULE LES BAS (ABA),2010-10-08,974
178,NaN,NaN,NaN,NaN,NaN,NaN,None,97414,SAINT-LOUIS,Ville,974000084,RESEAU ST-LOUIS VILLE (SUP.),2010-10-08,974
179,NaN,NaN,NaN,NaN,NaN,NaN,None,97414,SAINT-LOUIS,Bon Accueil,974000086,RESEAU LES MAKES (SUP.),2010-10-08,974
180,NaN,NaN,NaN,NaN,NaN,NaN,None,97414,SAINT-LOUIS,Le Gol,974000554,RESEAU MAISON ROUGE (SUP.),2010-10-08,974
181,NaN,NaN,NaN,NaN,NaN,NaN,None,97416,SAINT-PIERRE,"Centre ville, amont rue du Presbytère",974000103,RESEAU VILLE SAPHIR (SUP.),2010-10-08,974
182,NaN,NaN,NaN,NaN,NaN,NaN,None,97418,SAINTE-MARIE,La Découverte,974000757,RESEAU PARNY (ABA),2010-10-08,974
183,NaN,NaN,NaN,NaN,NaN,NaN,None,97418,SAINTE-MARIE,Quartier Moka,974003679,NOUVEAU RESEAU MOKA,2016-04-06,974
184,NaN,NaN,NaN,NaN,NaN,NaN,None,97422,TAMPON (LE),"Bras de Pontho, Dassy",974003697,RESEAU BRAS PONTHO DASSY,2019-11-04,974


,gid,cn_udi,locked,_userid_creation,_userid_modification,_edit_datemaj,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept
114,115.0,974000761,None,None,None,None,"MULTIPOLYGON (((55.55597 -21.34763, 55.55722 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,126.0,974000977,None,None,None,None,"MULTIPOLYGON (((55.51695 -20.93690, 55.51230 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,135.0,974001543,None,None,None,None,"MULTIPOLYGON (((55.55623 -20.89715, 55.55648 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,156.0,974000136,None,None,None,None,"MULTIPOLYGON (((55.56023 -21.21197, 55.55999 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,171.0,974000640,None,None,None,None,"MULTIPOLYGON (((55.53758 -21.02300, 55.53941 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


,gid,cn_udi,locked,_userid_creation,_userid_modification,_edit_datemaj,geometry,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,cddept


In [13]:
data_AtlaSante.convert_dtypes(convert_string=False).to_csv('outputs/tableau_AtlaSante.csv')
data_AtlaSante.convert_dtypes(convert_string=False)

,Année,Champ AtlaSante,Champ data.gouv,Fichier,UDIs AtlasSante,UDIs data.gouv,Union,Intersection,Pas dans AtlaSante,Pas dans data.gouv,Pas dans dept
Région,,,,,,,,,,,
Auvergne-Rhône-Alpes,2020,c_code_udi,cdreseau,data/AtlaSante_Auvergne_Rhone_Alpes_2020/ars_r...,5399,5260,5489,5170,90,229,0
Bourgogne-Franche-Comté,2019,code,cdreseau,data/AtlaSante_Bourgogne_Franche_Comte_2019/ar...,2087,1963,2123,1927,36,160,0
Bretagne,2017,c_ins_code,cdreseau,data/AtlaSante_Bretagne_2017/ars_r53_bretagne_...,719,688,733,674,14,45,0
Centre-Val de Loire,2020,code,cdreseau,data/AtlaSante_Centre_Val_de_Loire_2020/ars_r2...,1085,1065,1091,1059,6,26,0
Corse,2017,c_cn_udi,cdreseau,data/AtlaSante_Corse_2017/ars_r94_udi_j.json,630,615,631,614,1,16,0
Normandie,2020,codeudi,cdreseau,data/AtlaSante_Normandie_2020/ars_r28_udi_2020...,1004,1007,1014,997,10,7,0
Nouvelle-Aquitaine,2018,c_code_udi,cdreseau,data/AtlaSante_Nouvelle-Aquitaine_2018/ars_r75...,1896,1824,2131,1589,235,307,0
Occitanie,2020,code_n_udi,cdreseau,data/AtlaSante_Occitanie_2020/ars_r76_udi_2020...,4063,3714,4115,3661,54,402,0
Provence-Alpes-Côte d'Azur,2020,code_infof,cdreseau,data/AtlaSante_PACA_2019/ars_r93_udi_infofactu...,1373,1700,1889,1184,516,189,0
